# Channel Break Out Detection

## Import Data

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

In [3]:
#df = pd.read_csv(r"D:\CARLOS\TRADING\EURUSD_Candlestick_1_D_BID_04.05.2003-21.01.2023.csv")
file_path = r'D:\traderxpro\data\df_h.txt'
#df = pd.read_csv(file_path, sep='\t', dtype={'open':float, 'high':float, 'low':float, 'close':float, 'volume':float,'fecha':datetime, 'companyName':str, 'fecha2:':datetime})
data= pd.read_csv(file_path, sep='\t')
data['fecha']=pd.to_datetime(data['fecha'])
data['fecha2']=pd.to_datetime(data['fecha2'])

In [5]:
data=data.rename(columns={"open": "Open", "close": "Close", "high": "High", "low": "Low", "fecha2":"date", "volume":"Volume"})
data.head(50)

,fecha,Open,High,Low,Close,Volume,companyName,date
0,2024-10-16 20:00:00,582.3000,582.3000,582.3000,582.3000,1502080,SPY,2024-10-16 21:00:00
1,2024-10-16 19:00:00,581.8600,581.9000,579.7800,581.5800,30881,SPY,2024-10-16 20:00:00
2,2024-10-16 18:00:00,582.0000,582.3000,581.7000,581.8700,1546066,SPY,2024-10-16 19:00:00
3,2024-10-16 17:00:00,581.9600,601.0618,581.9000,581.9600,44443,SPY,2024-10-16 18:00:00
4,2024-10-16 16:00:00,582.2400,582.3415,580.8956,581.9300,6564790,SPY,2024-10-16 17:00:00
5,2024-10-16 15:00:00,582.4200,582.6500,582.1000,582.2400,8202864,SPY,2024-10-16 16:00:00
6,2024-10-16 14:00:00,581.6300,582.8300,581.4800,582.4000,3234392,SPY,2024-10-16 15:00:00
7,2024-10-16 13:00:00,581.4000,581.6900,580.9350,581.6350,2290796,SPY,2024-10-16 14:00:00
8,2024-10-16 12:00:00,580.2350,581.4000,580.2000,581.3900,1871979,SPY,2024-10-16 13:00:00
9,2024-10-16 11:00:00,580.1200,580.9000,579.7800,580.2400,3564060,SPY,2024-10-16 12:00:00


In [7]:
data['hour'] = data['date'].dt.hour
df=data.query("hour>=9 and hour<=16").sort_values(by=["date"],ascending=True)

In [9]:
df=df.reset_index()
df=df.drop(['index'], axis=1)

In [11]:
df[913:920]

,fecha,Open,High,Low,Close,Volume,companyName,date,hour
913,2023-04-18 09:00:00,415.7900,415.810,414.4900,414.6750,5729247,SPY,2023-04-18 10:00:00,10
914,2023-04-18 10:00:00,414.6800,414.705,412.9300,413.3150,8802654,SPY,2023-04-18 11:00:00,11
915,2023-04-18 11:00:00,413.3100,413.835,412.7802,413.2200,5292848,SPY,2023-04-18 12:00:00,12
916,2023-04-18 12:00:00,413.2400,414.250,413.1100,413.7900,4279822,SPY,2023-04-18 13:00:00,13
917,2023-04-18 13:00:00,413.7915,414.460,413.5600,413.9512,4933038,SPY,2023-04-18 14:00:00,14
918,2023-04-18 14:00:00,413.9550,414.490,413.0800,413.8313,6447222,SPY,2023-04-18 15:00:00,15
919,2023-04-18 15:00:00,413.8301,414.595,413.8200,414.2200,11836673,SPY,2023-04-18 16:00:00,16


In [13]:
data.query("date>='2024-05-31' and date<='2024-06-04'")

,fecha,Open,High,Low,Close,Volume,companyName,date,hour
1594,2024-06-03 20:00:00,527.8000,527.8000,527.8000,527.800,978354,SPY,2024-06-03 21:00:00,21
1595,2024-06-03 19:00:00,527.8400,527.9700,527.5200,527.560,20362,SPY,2024-06-03 20:00:00,20
1596,2024-06-03 18:00:00,527.8000,528.0000,527.6900,527.840,994710,SPY,2024-06-03 19:00:00,19
1597,2024-06-03 17:00:00,527.9200,527.9500,527.6999,527.800,74951,SPY,2024-06-03 18:00:00,18
1598,2024-06-03 16:00:00,527.7600,529.0100,525.8042,527.870,6747108,SPY,2024-06-03 17:00:00,17
1599,2024-06-03 15:00:00,525.7500,527.8300,525.5600,527.770,8781478,SPY,2024-06-03 16:00:00,16
1600,2024-06-03 14:00:00,525.2200,526.3120,524.6700,525.750,3787075,SPY,2024-06-03 15:00:00,15
1601,2024-06-03 13:00:00,523.7700,526.9661,522.6000,525.190,4193018,SPY,2024-06-03 14:00:00,14
1602,2024-06-03 12:00:00,526.0900,526.4000,523.2100,523.790,5824448,SPY,2024-06-03 13:00:00,13
1603,2024-06-03 11:00:00,526.0328,526.6900,525.4000,526.090,4880520,SPY,2024-06-03 12:00:00,12


In [15]:
df['SMA20'] = df['Close'].rolling(20).mean()
df.dropna(inplace=False)
df['SMA40'] = df['Close'].rolling(40).mean()
df.dropna(inplace=False)
df['SMA100'] = df['Close'].rolling(100).mean()
df.dropna(inplace=False)
df['SMA200'] = df['Close'].rolling(200).mean()

In [17]:
backcandles = 45

## Detect Pivots/Fractals

In [20]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [22]:
window=3
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

## Detect Price Channel

In [25]:
def collect_channel(candle, backcandles, window):
    localdf = df[candle-backcandles-window:candle-window]
    #localdf['isPivot'] = localdf.apply(lambda x: isPivot(x.name,window), axis=1)
    highs = localdf[localdf['isPivot']==1].High.values
    idxhighs = localdf[localdf['isPivot']==1].High.index
    lows = localdf[localdf['isPivot']==2].Low.values
    idxlows = localdf[localdf['isPivot']==2].Low.index
    
    if len(lows)>=3 and len(highs)>=3:
        sl_lows, interc_lows, r_value_l, _, _ = stats.linregress(idxlows,lows)
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)
    
        return(sl_lows, interc_lows, sl_highs, interc_highs, r_value_l**2, r_value_h**2)
    else:
        return(0,0,0,0,0,0)

In [27]:
df['Channel'] = [collect_channel(candle, backcandles, window) for candle in df.index]

## Detect Break Out

In [62]:
def isBreakOut(candle, backcandles, window):
    if (candle-backcandles-window)<0:
        return 0
    
    sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h = df.iloc[candle].Channel
    
    prev_idx = candle-1
    prev_high = df.iloc[candle-1].High
    prev_low = df.iloc[candle-1].Low
    prev_close = df.iloc[candle-1].Close
    
    curr_idx = candle
    curr_high = df.iloc[candle].High
    curr_low = df.iloc[candle].Low
    curr_close = df.iloc[candle].Close
    curr_open = df.iloc[candle].Open

    SMA40=df.iloc[candle].SMA40
    SMA20=df.iloc[candle].SMA20

    # if ( prev_high > (sl_lows*prev_idx + interc_lows) and
    #     prev_close < (sl_lows*prev_idx + interc_lows) and
    #     curr_open < (sl_lows*curr_idx + interc_lows) and
    #     curr_close < (sl_lows*prev_idx + interc_lows) and
    #     SMA40>SMA20): #and r_sq_l > 0.9
    #     return 1
    
    if ( prev_low < (sl_highs*prev_idx + interc_highs) and
        prev_close > (sl_highs*prev_idx + interc_highs) and
        curr_open > (sl_highs*curr_idx + interc_highs) and
        curr_close > (sl_highs*prev_idx + interc_highs) and 
        SMA40>SMA20 and
        SMA40>=curr_open): #and r_sq_h > 0.9
        return 2
    
    else:
        return 0

In [64]:
df["isBreakOut"] = [isBreakOut(candle, backcandles, window) for candle in df.index]

In [65]:
def SIGNAL():
    return df.isBreakOut

In [66]:
from backtesting import Strategy
from backtesting import Backtest
from backtesting.test import SMA
import backtesting

class BreakOut(Strategy):
    initsize = 0.1
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        close = self.data.Close
        self.sma1 = self.I(SMA, close, 20)
        self.sma2 = self.I(SMA, close, 40)

    def next(self):
        super().next()
        TPSLRatio = 1.2

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Low[-2]
            tp1 = self.data.Close[-1] + abs(self.data.Close[-1]-sl1)*TPSLRatio
            #tp2 = self.data.close[-1] + abs(self.data.Close[-1]-sl1)*TPSLRatio/3
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            #self.buy(sl=sl1, tp=tp2, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.High[-2]
            tp1 = self.data.Close[-1] - abs(sl1-self.data.Close[-1])*TPSLRatio
            #tp2 = self.data.close[-1] - abs(sl1-self.data.Close[-1])*TPSLRatio/3
            self.sell(sl=sl1, tp=tp1, size=self.mysize, limit=100)
            #self.sell(sl=sl1, tp=tp2, size=self.mysize)


bt = Backtest(df, BreakOut, cash=1000, margin=1/50, commission=.00)
stat = bt.run()
stat

C:\Users\carlo\AppData\Local\Temp\ipykernel_7276\2083877719.py:35: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, BreakOut, cash=1000, margin=1/50, commission=.00)


Start                                     0.0
End                                    3935.0
Duration                               3935.0
Exposure Time [%]                    2.896341
Equity Final [$]                   1064.13318
Equity Peak [$]                    1182.19018
Return [%]                           6.413318
Buy & Hold Return [%]               49.158448
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.070595
Avg. Drawdown [%]                   -4.339408
Max. Drawdown Duration                  662.0
Avg. Drawdown Duration             200.222222
# Trades                                 21.0
Win Rate [%]                        42.857143
Best Trade [%]                       2.060243
Worst Trade [%]                     -1.843552
Avg. Trade [%]                    

In [70]:
bt.plot()

C:\Users\carlo\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\carlo\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p3207', ...)

In [ ]:
df[913:920]

In [ ]:
df.query("isBreakOut==2")
#df.iloc[4260:4270]

In [ ]:
df =df.reset_index()

In [ ]:
df = df.drop("Gmt time", axis=1)